100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  9.30it/s]


In [ ]:
s_time_interv= total_training_data[['Subject','Session','time_stamp']]
s_time_interv=s_time_interv.groupby(['Subject','Session']).agg({'time_stamp': [np.min,np.max]})
s_time_interv.columns=s_time_interv.columns.droplevel()
s_time_interv['time_interval']=s_time_interv['amax']-s_time_interv['amin']
s_time_interv['time_interval']=s_time_interv['time_interval'].dt.total_seconds()
s_time_interv=s_time_interv[['time_interval']]
s_time_interv=s_time_interv.unstack()
s_time_interv.columns=s_time_interv.columns.droplevel()
#s=s.reset_index(drop=True)

In [ ]:
s_time_interv.plot(kind='bar',ylabel='time_interval')

In [ ]:
s_label_dist= total_training_data[['Subject','labels','time_stamp']]
s_label_dist=s_label_dist.groupby(['Subject','labels']).count()
s_label_dist=s_label_dist.unstack()
s_label_dist.columns=s_label_dist.columns.droplevel()
s_label_dist.plot(kind='bar',xlabel='Subject',ylabel='count')

In [ ]:
s_label_dist= total_training_data[['Session','labels','time_stamp']]
s_label_dist=s_label_dist.groupby(['Session','labels']).count()
s_label_dist=s_label_dist.unstack()
s_label_dist.columns=s_label_dist.columns.droplevel()
s_label_dist.plot(kind='bar',xlabel='Session',ylabel='count')

In [ ]:
s_label_dist

In [ ]:
s_label_dist.reset_index().plot.bar()

In [ ]:
new_df = total_training_data[['time_stamp','labels','measurements']].groupby([pd.Grouper(key='time_stamp',freq='5min'),'labels']).count().reset_index().pivot(columns=['labels','time_stamp'])
#.unstack()
#new_df.columns = new_df.columns.droplevel()
#new_df=new_df.reset_index()
# new_df.plot.bar()

In [6]:
X_total=np.vstack(total_training_data.measurements)
Y_total=np.vstack(total_training_data.labels)
Y_total=Y_total.reshape((len(Y_total)))

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_total, Y_total, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [9]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)


In [ ]:
y_pred = rf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)

In [ ]:
from sklearn import tree
X = X_train
Y = y_train
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)


In [ ]:
test_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, test_pred)

In [ ]:
accuracy

In [ ]:
##############Predictions_Test_set###################

In [ ]:
test_files=glob.glob('TestData/*.csv')

In [ ]:
Subject_list= [f.split('_')[1] for f in test_files]
Session_list= [f.split('_')[2] for f in test_files]
x_sensor_dat=pd.DataFrame([])
y_sensor_dat=pd.DataFrame([])
x_time_dat=[]
for sub,sess in zip(Subject_list,Session_list):
    #x_sensor_dat=pd.DataFrame(x_sensor_dat.append(pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x.csv",header=None)))
    
    x_sensor_dat=pd.concat([pd.read_csv(f"TestData/subject_{sub}_{sess}__x.csv",header=None),pd.read_csv(f"TestData/subject_{sub}_{sess}__x_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1)

    x_sensor_dat['measurements']=x_sensor_dat.apply(lambda x: [x[0],x[1],x[2],x[3],x[4],x[5]],axis=1)
    x_sensor_dat=x_sensor_dat[['time_stamp','measurements']]
    x_sensor_dat["time_stamp"]=pd.to_datetime(x_sensor_dat["time_stamp"],unit='s').round('us')
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    x_sensor_dat['Subject']=int(sub)
    x_sensor_dat['Session']=int(sess)
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')

In [ ]:
X_test=np.vstack(x_sensor_dat.measurements)
#Y_total=np.vstack(total_training_data.labels)
#Y_total=Y_total.reshape((len(Y_total)))

In [ ]:
X_test.shape

In [ ]:
test_pred = rf.predict(X_test)
#accuracy = accuracy_score(y_val, y_pred)

In [ ]:
test_pred.shape

In [ ]:
x_sensor_dat['test_pred']=list(test_pred)

In [ ]:
x_sensor_dat

In [ ]:
x_sensor_dat.groupby('test_pred')[['Session']].count()